In [1]:
import sys
import matplotlib as mpl
import hyperspy.api as hs
from pathlib import Path as path
import numpy as np
import pandas as pd
import xarray as xr
import datetime

import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.gridspec as gridspec
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap, to_rgba

In [2]:
# 画图的初始设置
plt.style.use(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-python\Figure\liuchzzyy.mplstyle')
# display(plt.style.available)

# 颜色设定
sys.path.append(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Python\Figure')
from colors import tol_cmap, tol_cset
colors = list(tol_cset('vibrant'))
if r'sunset' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('sunset'))
if r'rainbow_PuRd' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('rainbow_PuRd')) # 备用 plasma

# 输出的文件夹
path_out = path(r"C:\Users\chengliu\Desktop\Figure")

In [3]:
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
def add_sizebar(ax, size, data, color):
    asb = AnchoredSizeBar(ax.transData,
                          size / data.axes_manager["y"].scale,
                          '{} {}'.format(size, data.axes_manager['y'].units),
                          loc='lower left',
                          pad=0.1, borderpad=0.5, sep=1.0, #0.5
                          frameon=False,
                          color=color,
                         label_top=True)
    ax.add_artist(asb)

def remove_bkg(data, ll, energy_limit, mask=None, plot_fig=False):
    %matplotlib ipympl
    hs.set_log_level('ERROR')
    model_bkg = data.create_model(low_loss=ll, auto_add_edges=False, auto_background=True)
    model_bkg.fit_component(model_bkg.components.PowerLaw, signal_range=(energy_limit[0], energy_limit[1]), fit_independent=True, only_current=True,)
    model_bkg.assign_current_values_to_all(components_list = [model_bkg.components.PowerLaw])
    model_bkg.fit_component(model_bkg.components.PowerLaw, signal_range=(energy_limit[0], energy_limit[1]), mask=mask, fit_independent=True, only_current=False,)
    
    if plot_fig == True:
       model_bkg.plot(plot_components=True) 
        
    return  (data - model_bkg.as_signal(component_list=['PowerLaw'])).deepcopy()

def transparent_single_color_cmap(color0, color1):
    """Return a single color matplotlib cmap with the transparency increasing
    linearly from 0 to 1."""
    return LinearSegmentedColormap.from_list("", [to_rgba(color0, 1), to_rgba(color1, 1)])

### EELS Mapping

In [ ]:
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\TEM\ExSitu\αMnO2\Charge\1st1.53V\αMnO2 + PVDF + SP\1M ZnSO4 + 0.2M MnSO4\2024-EMCA\EELS\Results\SI 1_630mm_5mm_0.9eVCh_4nm_15ms_150pA\EELS Map')
file_O = path.joinpath(path_file, r'O K Signal (EELS).dm4')
file_Mn = path.joinpath(path_file, r'Mn L Signal (EELS).dm4')
file_S = path.joinpath(path_file, r'S K Signal (EELS).dm4')
file_Zn = path.joinpath(path_file, r'Zn L Signal (EELS).dm4')

file_rgb = path.joinpath(path_file, r'RGB Composite.dm4')
file_line = path.joinpath(path_file, r'Profiles of RGB Composite.dm4')
line = hs.load(file_line)

data = hs.load([file_O, file_Mn, file_S, file_Zn, file_rgb])
data

In [ ]:
for i in range(len(data)):
    data[i].axes_manager.convert_units(axes="signal", units='nm', same_units=True, factor=1000)
    data[i].axes_manager.convert_units(axes="signal", units='nm', same_units=True, factor=1000)
data[0].axes_manager

In [ ]:
# TEM 图
%matplotlib inline

plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.imshow(data[0].data, cmap=transparent_single_color_cmap('k', (0/255, 255/255, 0/255)), aspect=1.0, alpha=1.0) # O

add_sizebar(ax, 100, data[0], 'w')
remove_spine(ax)

ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

plt.savefig(path.joinpath(path_out, f'{file_O.stem}_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, f'{file_O.stem}_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

In [ ]:
# TEM 图
%matplotlib inline

plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.imshow(data[1].data, cmap=transparent_single_color_cmap('k', (255/255, 255/255, 0/255)), aspect=1.0, alpha=1.0) # Mn

add_sizebar(ax, 100, data[1], 'w')
remove_spine(ax)

ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

plt.savefig(path.joinpath(path_out, f'{file_Mn.stem}_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, f'{file_Mn.stem}_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

In [ ]:
# TEM 图
%matplotlib inline

plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.imshow(data[2].data, cmap=transparent_single_color_cmap('k', (0/255, 255/255, 255/255)), aspect=1.0, alpha=1.0) # S

add_sizebar(ax, 100, data[2], 'w')
remove_spine(ax)

ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

plt.savefig(path.joinpath(path_out, f'{file_S.stem}_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, f'{file_S.stem}_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

In [ ]:
# TEM 图
%matplotlib inline

plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.imshow(data[3].data, cmap=transparent_single_color_cmap('k', (255/255, 128/255, 255/255)), aspect=1.0, alpha=1.0) # Zn

add_sizebar(ax, 100, data[3], 'w')
remove_spine(ax)

ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

plt.savefig(path.joinpath(path_out, f'{file_Zn.stem}_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, f'{file_Zn.stem}_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

In [ ]:
data[4].data['A'].min()

In [ ]:
# 转换为 float 格式并归一化
rgba_data = np.zeros(data[4].data.shape + (3,), dtype=np.float32)
rgba_data[..., 0] = data[4].data['R'] / 255.0  # Red channel
rgba_data[..., 1] = data[4].data['G'] / 255.0  # Green channel
rgba_data[..., 2] = data[4].data['B'] / 255.0  # Blue channel
# rgba_data[..., 3] = data[4].data['A'] / 255.0  # Alpha channel

In [ ]:
# line 图
%matplotlib inline

plt.close('all')
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1], height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])
ax.imshow(rgba_data, cmap=transparent_single_color_cmap('k', (255/255, 128/255, 255/255)), aspect=1.0, alpha=1.0)

add_sizebar(ax, 100, data[3], 'w')
remove_spine(ax)

ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])

ax.plot(line[0].data, c=(0/255, 255/255, 0/255), label=r'O')
ax.plot(line[1].data, c=(255/255, 255/255, 0/255), label=r'Mn')
ax.plot(line[2].data, c=(0/255, 255/255, 255/255), label=r'S')
ax.plot(line[3].data, c=(255/255, 128/255, 255/255), label=r'Zn')


plt.savefig(path.joinpath(path_out, f'{file_line.stem}_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, f'{file_line.stem}_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### 对齐对应的图片

In [ ]:
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\TEM\ExSitu\αMnO2\Charge\1st0.9V\αMnO2 + PVDF + SP\1M ZnSO4 + 1M MnSO4\2024-EMCA\TEM\Data')
file1 = path.joinpath(path_file, r'0010 - B8_Ceta_165_kx.emd')
file6 = path.joinpath(path_file, r'0015 - B8_Ceta_520_kx.emd')
file2 = path.joinpath(path_file, r'0016 - B8_Ceta_265_kx.emd')
file3 = path.joinpath(path_file, r'0017 - B8_Ceta_210_kx.emd')
file4 = path.joinpath(path_file, r'0018 - B8_Ceta_520_kx.emd')
file5 = path.joinpath(path_file, r'0019 - B8_Ceta_520_kx.emd')

data = hs.load([file1,file2,file3,file4,file5,file6])
data

In [ ]:
img.shape

In [ ]:
# 画图
from scipy import ndimage
fig = plt.figure(figsize=(7, 5))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1,], height_ratios=[1,1], wspace=0.0, hspace=0.0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0.0, 0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[3].data/data[3].data.max().max(), 0, reshape=True)
ax.imshow(img, cmap='gray', aspect=1.0, vmin=0.0, vmax=0.75)
add_sizebar(ax, 10, data[3], 'w')
remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0.02, 0.0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[4].data/data[4].data.max().max(), 0, reshape=True)
ax.imshow(img, cmap='gray', aspect=1.0, vmin=0.0, vmax=0.7)
add_sizebar(ax, 10, data[4], 'w')
remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0.04, 0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[2].data/data[2].data.max().max(), 0, reshape=True)
ax.imshow(img[400:-850,400:-850], cmap='gray', aspect=1.0, vmin=0.0, vmax=0.85)
add_sizebar(ax, 10, data[2], 'w')
remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)


# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=1)
ax = subfig.add_subplot()
ax.set_position([0.0, 0.0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[0].data/data[0].data.max().max(), 90, reshape=True)
ax.imshow(img[:-2900, 2900:], cmap='gray', aspect=1.0, vmin=0, vmax=0.75)
add_sizebar(ax, 10, data[0], 'w')

remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0.02, 0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[1].data/data[1].data.max().max(), 0, reshape=True)
ax.imshow(img[600:-500, 600:-500], cmap='gray', aspect=1.0, vmin=0.0, vmax=0.15)
add_sizebar(ax, 10, data[1], 'w')
remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

# 图 F
subfig = fig.add_subfigure(gs[1, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0.04, 0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[5].data/data[5].data.max().max(), 0, reshape=True)
ax.imshow(img, cmap='gray', aspect=1.0, vmin=0.0, vmax=0.7)
add_sizebar(ax, 10, data[5], 'w')
remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

plt.savefig(path.joinpath(path_out, r'300_PaperUno_Figure_5.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300)
plt.savefig(path.joinpath(path_out, r'600_PaperUno_Figure_5.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600)

plt.show()

### 对齐 TEM 和 EELS

In [ ]:
file1 = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\TEM\ExSitu\αMnO2\Charge\1st0.9V\αMnO2 + PVDF + SP\1M ZnSO4 + 1M MnSO4\2024-EMCA\TEM\Data\0011 - B8_Ceta_165_kx.emd')
file2 = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\TEM\ExSitu\αMnO2\Charge\1st0.9V\αMnO2 + PVDF + SP\1M ZnSO4 + 1M MnSO4\2024-EMCA\EELS\Data\SI2_80pA_10ms\Zn.dm4')
data = hs.load([file1,file2])
data

#### Energy axis 进行插值

In [ ]:
hs.set_log_level('ERROR')
from hyperspy.axes import UniformDataAxis
new_scale = data[0].axes_manager['x'].scale
axis_x = UniformDataAxis(offset=0, scale=new_scale, size=((data[1].axes_manager['x'].high_value - data[1].axes_manager['x'].low_value)//new_scale+1), name="x", units='nm', navigate=False, is_binned=False)
axis_y = UniformDataAxis(offset=0, scale=new_scale, size=((data[1].axes_manager['y'].high_value - data[1].axes_manager['y'].low_value)//new_scale+1), name="y", units='nm', navigate=False, is_binned=False)

data[1].interpolate_on_axis(axis_x, 0, inplace=True)
data[1].interpolate_on_axis(axis_y, 1, inplace=True)

data[1].axes_manager

In [ ]:
# 画图
from scipy import ndimage
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1], height_ratios=[1], wspace=0.0, hspace=0.0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0.0, 0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[0].data/data[0].data.max().max(), -90, reshape=True)
ax.imshow(img, cmap='gray', aspect=1.0, vmin=0.0, vmax=1.0)
add_sizebar(ax, 20, data[0], 'k')
# remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.3, 0.0, 1.0, 1.0])
ax.set_box_aspect(1.0)
img = ndimage.rotate(data[1].data/data[1].data.max().max(), 0, reshape=True)
ax.imshow(img[100:-300, 1535:-1500], cmap='gray', aspect=1.0, vmin=-0.1, vmax=1.0)
add_sizebar(ax, 20, data[1], 'r')
# remove_spine(ax)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False,)

plt.savefig(path.joinpath(path_out, r'300_TEM_2.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=300)
plt.savefig(path.joinpath(path_out, r'600_TEM_2.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600)

plt.show()